# --- Day 4: Ceres Search ---

"Looks like the Chief's not here. Next!" One of The Historians pulls out a device and pushes the only button on it. After a brief flash, you recognize the interior of the Ceres monitoring station!

As the search for the Chief continues, a small Elf who lives on the station tugs on your shirt; she'd like to know if you could help her with her word search (your puzzle input). She only has to find one word: XMAS.

This word search allows words to be horizontal, vertical, diagonal, written backwards, or even overlapping other words. It's a little unusual, though, as you don't merely need to find one instance of XMAS - you need to find all of them. Here are a few ways XMAS might appear, where irrelevant characters have been replaced with .:
```
..X...
.SAMX.
.A..A.
XMAS.S
.X....
```
The actual word search will be full of letters instead. For example:
```
MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX
```
In this word search, XMAS occurs a total of 18 times; here's the same word search again, but where letters not involved in any XMAS have been replaced with .:
```
....XXMAS.
.SAMXMS...
...S..A...
..A.A.MS.X
XMASAMX.MM
X.....XA.A
S.S.S.S.SS
.A.A.A.A.A
..M.M.M.MM
.X.X.XMASX
```
Take a look at the little Elf's word search. How many times does XMAS appear?

# thoughts


this is a graph process, and we can try to simplify but for now I think we should just broot force it...


from point p there are only 6 directions to check:        
```
(x+-1, y+-1) (x+0, y+-1) (x+1, y+-1)
(x+-1, y+0 ) (x+0, y+0 ) (x+1, y+0 )
(x+-1, y+1 ) (x+0, y+1 ) (x+1, y+1 )
``` 
and this all goes with `xmas` goes to max is 4 so so `0:3` but we cant look at x-1,y-3 so maybe we can make this eiaser and just enumerate all of these in one pass.

maybe we can create the sub strings and then check for xmas either forward or backwards but that will be the same as checking it in place. 

```
(-1,-1)(0,-1)(+1,+1)
(-1, 0)(0,0 )(+1,0)
```

```
vectors are 
```
[-l,-l][0, -l][+l,-l]
[-l, 0][.....][+l, 0]
[-l,+l][0, +l][+l,+l]
```
nw n ne
 w   e 
sw s se


In [ ]:
test_text = """MMMSXXMASM
MSAMXMSMSA
AMXSXMAAMM
MSAMASMSMX
XMASAMXAMM
XXAMMXXAMA
SMSMSASXSS
SAXAMASAAA
MAMMMXMMMM
MXMXAXMASX"""


def test_wordsearch(f):
    assert f(test_text) == 18


In [ ]:
import numpy as np


def find_xmas(x, y, message, x_max, y_max):
    out = []
    instructions = {
        "e": [[0, 0], [0, 1], [0, 2], [0, 3]],
        "w": [[0, 0], [0, -1], [0, -2], [0, -3]],
        "s": [[0, 0], [1, 0], [2, 0], [3, 0]],
        "n": [[0, 0], [-1, 0], [-2, 0], [-3, 0]],
        "sw": [[0, 0], [1, -1], [2, -2], [3, -3]],
        "ne": [[0, 0], [-1, 1], [-2, 2], [-3, 3]],
        "nw": [[0, 0], [-1, -1], [-2, -2], [-3, -3]],
        "se": [[0, 0], [1, 1], [2, 2], [3, 3]],
    }

    for i, v in instructions.items():
        if x < 3 and "n" in i:  # cant go west
            continue
        if x_max - x <= 3 and "s" in i:  # cant co east
            continue
        if y < 3 and "w" in i:
            continue
        if y_max - y <= 3 and "e" in i:
            continue
        res = "".join([message[x + v[j][0], y + v[j][1]] for j in range(4)])

        # print(f'{x},{y},{i},{v},{message[x,y]},{res}')
        out.append(res)

    return out


def gen_matrix(text):
    rows = text.splitlines()
    tem = []
    for i in [r for r in rows if r]:
        tem.append([j for j in i])
    return np.asmatrix(np.array(tem))


x = 1
y = 4


def check_xmas(text_input):
    # find_xmas()
    a = gen_matrix(text_input)
    s = a.shape
    x_max = s[0]
    y_max = s[1]
    res = []
    for x in range(x_max):
        for y in range(y_max):
            res.append(find_xmas(x, y, a, x_max, y_max))

    ## flatten
    # print(res)
    out = []
    [[out.append(j) for j in i] for i in res]
    res = out
    acc = 0
    for i in res:
        if i == "XMAS" or i == "SAMX":
            acc = acc + 1
    return int(acc / 2)


check_xmas(test_text)


18

In [ ]:
major_input = """SSSMXMMSMMXXXSMXMASXSAXXMSXSMSXMMSSMMMSMAMXMASASXSXMMSXXXMAXSMSXMAMSAMXMSMAMSAMAMMXSAMXMMSAMMMMSMMMSMMXMMSSMMXSMAMMAAMAMXSSXXXAMXXXSASMSSSSS
MAASXMASAMXMXMMMXMMMSSMAXSAMXSAXMASAAAXMAAMMAAASMSAMSMAMSMMMXXXAMMMXMXMXAMAMAXMAXXAMXMAMXMAMAASXMAAAMXAMXAAXSAASASXMSMSXMXMASXSASMMMAMAAAAAA
MSMMMMMSMMAAAAAMASAMMAXXXMAMASXMMXSXMMSSSSXMXMMMASAMXMAMAAASXSXMMSXAAAMSMSAXMMMSSMASASMSSSSMSXSASMSSXXASMXSMMMXMXMAAMXMASAMXMAXASXAMAMMMMMMM
AAAXAAASXSXSMSASASAXXAMXMSAMXSSMSMMXSXAAAMXMAMASXSAMXSASMSMSAMXMAXSXMAXAAXAMXAMAAXMMMSAAAAXAMASAMXAAASAMMXXAMXAMSXMMMASMMASXMMMAMXXMXSXXXAMX
SXSSMSXSAMAMXXXMMSASAMXAXAASAMXSAMXAXAMSMMASASASXMAMASASAMXMXMAMMXAMXSSMXMSASMMSSMXMXXMMMMMAMMMMMMMSMAAXSASAMMSXSAAMSAMASAMAAXMSMSSMSMMMSMMX
AAXMAMXMXMAMMSAXAMASMASMSSSMASXAXMXXSXXAASXMASAMMMSMASASMMSSSSXSXSASAMAASAMMSAAAAMAMXXXXMAMXSMXSAMXXXMAXMASMMAXASAMMMXSXMASMMMSAMAAMSAAMASXM
MAMSASAMAMXSAMXMXSAMXMAXXMAXXMAXSMXAMMXSXMXMMMMMAMAAMMXMAAAXAMXAASAMXMMMAAMXMMMMSSXSXXSSSXSASXAMAMASAXMSMXMAMAMAMAMXMASASAMXAMXASXXMMSMSASXA
AXMMMMAMXAAMASAMXMASXSMSSSMSMSMAAAXXMAAXAMASAAASXMXXMAMSMMSSSMSSXMXMSXMSMSSSXXXAXMXMMXSAAAMAMMMMXMMSMMMAMXSAMSMMMAMAMASXMXASXSMMMAMMXMXMAMMM
SXMAMSSMMMXSMMXMAMXMXASAAXAAMAMXAMXMMSXSASAXASMSXMSXMXAAXMAAAMAMXMXAXAXXXXAMAMAMMMMSASMMMXMAMAXSMSASAXMAXXMXSMAMSMSAMXSXMXMMMSMSAMSMAMXSMMXS
XASAMAMXXMAMXMSSXSASXMMMMMXMSXMXMMXSAAMSAMMSAAMXAAMAMXSMSSMSMMSSSMASMXMMAMMMXMASAMMMXSAXXXXXXMSAAMASASMSMXXMAXAMAMAMXAMAMXAAAMASMMAXASXXAMAA
SXSASMMMSSMMSMMAASAMAXSASAXMXXMAAAAMAMXMXMXAMMSSMMSMMAXMAXXXAAXAAXAXMAMAAXAAASXSXSAMXSXMXSXSMXMMSMXMAMAAXSXSXSXSXSMAMMSASMSMSMAMASXSXSAMMMMS
SASXMXAAMAXMAAMMMMXSXMSAMASMAASMMMSSSMXSAMXMSAXXMMAAAXSXSXMXMMMSMMMXSAMSMSSSXSXSASMSMSXMXSAAMSXXAMMMSMMMMXAXAMAMAMXXXMSAXAAAXMASXMXXAXMASAXA
MAMXMMMMSAMSSSMMXSAMXXMAMSMMMMXSAAAAAMAMAMXXMMXSASMSMSSMMASXAAAAAAAAXXMAXXXMASXMAMAMAMMMAMSMMSMSMSASAMMAMMMMMMAMAMSSMMMXMSMSMMMSAMXMMMXXSASA
MAMAMXMAMAMXAAXMAMASAXMAMXAXSXASMMMMMMASAMXAAXAXMAXAXAMAMAMSSMSSSMSSMMSMXMMMAMXSMMXMAAAXXXAMXXAAASXMAMSASAAAXSXSMSAXAAAMXXMAXXASAMAMSASMMMMM
SASAMASASAMMMMMMASXMMSSMXSAMXMMXSAXXXSASASAMXMMSXMMMMMSSMAMAMXAMAMAMXXASAAXMASMSAXXSMSSSMXSMMMSMMMMSSMSASXSSMMASXMMSSMSMSAMMMMMMAMAXAAXXAXAX
SXSMSMSXSXMSASXSAXAAAMAAXMSMXASASXSMAMAMAMXMASAAMXASAMAXSAMAMAMSXMASXSASMSXSXMAXMMMAAXAAXAMAMAMMXMXAMXMAMMMMAXAMAAXMAMAASAMAXAAXMMMSMMMSXSXS
SXMAAAMAMXXXAXAMASXMXMAMXXAXXSMASAAMXMAMXMMSAMSSXSXXASMMSMSMSSXMMSASXMXMXXAXAMAMSMSASMMMMXMAMXSMAMMMSAMAMXASXMAMAMXXMMSMSMMMSMSAAAAAMXAMAXAM
MAMSMSMMSSSMSMAMMMAAAMSXSSMSXXMAMMMXXMASXAAMMMAXMXXSMMXAMXMAMMASAMMSMSASAMMMMMXSAAXMMAAMXSXSSMXMAMMXSXSXXMMAXXSMSMSSMAMAXXAXAAXMSMSMSMMSSMAM
MAMXAXXXAAAAAMMMSSMMMMMAMXXSXMMSSXSAMXAMMMMSSSMMMAASASMXSAMAMXAMMMMXAXAMXSXAAMXMMSMMSSMMASXAXXAXMSMAMXMXAXSXSXMAAAAAMAMMMXMSSMMXMAMASAXAXAAA
SASMMMSMMSMMMSSXMAXSAAMAMMAMAXXMAXAXMMASMSSXAXXAMAMSAMXAXASMSMMSXAXMSMXMXMMXMXAMXAXXAAXMASMAMSMMSAMASAAXMMSAMASXMMMMMXSXSXMAAXXXMAMXMXMASMXS
SXXAXAXAXXASXAAASMMSMSSSMMASXMMSSMSXSAXMASAMXMMMSXXMAMMMXMMAAXAMMXXAXAAMXMASXMASMMMSSSMMASAAAXAXXAMSSMSXXAMAMMMAXXMASMMASAMSSMMMMSSMAXMXSAMX
SXSMMXSSMSAMMMSMMAMXMMAMAMASXAMMMAAAXXXXXMXSXMMAXMASAMXAAMMMMMXSASMMMMXSXAMAMSAMASAMXXAMMMXSXSMASXMAXAXMMMXAMAXSMMSASAXASMMAAAAXAAMXMASAMASM
XXAAMXMMAMXMAMAASMMSXMXSXMASMSMAMMMSMMMXMMAMMXAMSXASAMMSXSAMXMXMASXSAXSAMXMAXMASAMASXSMMSMMMMAAAAAMMMMAXAMXSSXXMAMMMSXMXSXMSSMMMMMMAMAMAXAMX
SMSAMASAMXASASMMMAAMXMAXAMXSXAMASAAXMAAAXMAMXMMSXMXSAMAAASXMAMSMMMXAXMAAMXMXMSAMXSAMXAMAMXAMSAMMMMMXASAMSMAMAMASAMAASAMXSAXAXAAMAASXSSSSMMSX
SAMMSAXSAMXSXSXASMMMAMASXMXSXMXAMMSSSSMMSSMMSASMMSMSMMXSAXXXAXAAAXSMXASAMXMMMMXSMSXSSSMSSMMXMXXAAXXSXSMAAMMSASMXASMMMASASMMXSMMSMXSAAXAXMMMX
MAMAMASAMXMXAXMASMXXASMAASXSASMMXAAMMMAAXMXAXMAXXMAMXAMXMAMSMSSXMXAXMMXXMAXAMXMMAMAMAAAMMXMAMMMXMXAMXMMSXSMMAMXSXMMMMAMASXMMAMAXAXMMMMAMMAMX
SAMAMMSXAAMMXMMMMMMSMSMSXSASAMASMMMSASMSMSMMXSSSSMMMMMXAMXXAXXAASXMXSAMASMSSMSMMSMAMXMASMASXSASAAMSMSXAMXSAMXMXMASASMSMMMAMMMSXSMMMXAAAASASA
SXSSSMMMSAMXASXAAAAAASXMASAMAMAMASAMXSAAAAASAMAAAAXMAMMSMMSMXMSMMAXAMASAMMAMXAAXXXXSXSAAXXSAXASMXMAAMMSSMMASMMMSMMAXSAAAXAMSMMAMMAXMAXMXSASM
XXAMXAAAAAMSMSSSMSXMXMAMAMASXMXSAMASXMXMSMAMMSMSMMSMXSAXAMXSAMASMXMMSAMMSMAMMMMMMSXMAMSSXXMAMXXMASMSMAXAASAMAAAAXMXMXSSMSMSMAMMMSMSAXXSAMAMX
SMSSSSSSMAMAAMAXAMXSAMAMXSXMAMAMMSMMMSAMXXSXXSAMXMAAAMASMMASXSASMAAAMAXSAMMSXMAMAAAMXMMMXAMXMSASAMXXMXSAMMMSSMSSSMAMMMXMAXAMSMSMXMMMASMASXMS
XAXAAXMAXAXMMMAMXMASAXXXMXAMXMMSASAAASASAAXXSMMMSSMSMAAMAMASXMXMMXMAMXMMASAAASAMXSSMAMAAAXXAAXAMXSAMXXMASMAAMAAXAXXXASASMXMMMAMAMXMAMASMMMMX
MSMMMMSMMMSSSMXSAMXMAAMSSSXSASXMAXSMMXXMASMMXAAAXAXAXMXSXSXMXXXAAXAMSMSMSMMMXSAXAXMAMSMMSMSASMAMAMXSSMMAMXMASMMSSMSMAMMMXXMAMSMSSMSXXMAXAAMX
MAMXAAXXMSAAXAXSXSAMXSAAMAAXASMMMMMMXSSXXAAXASMSSMMAXXMMASAXXMMSMMMXAAAXAAXXAXAMSSXSXSAMXAMAXXSMASASASMXSXXXXAXXAAAMXMSMSASXSAAAAAAASXSMXMXM
SAMXMMSAMMMXMMMMMSXSAMMXMMMMSMXAMAXSAXMASXSXMMAXXXMSXXASAMAMXXAAAAAMMSMXSXMMMXAMXAXMMSAMMSMAMMXSASMSAMXMAXMMSMMSMSMSAAAAXMASMMSMMMXSAAMASXSM
MASXAMMAXXXXAAAMAMAMXMAMXAAXMASMSSXMXSXXMXMAMXXMSXMAASMMXSAXXMSSSMSXXAAAMXMASXSXMMXMAXMMSAMXXXAMXSAMXMMXMASAAAXMAMASXMMXMXMAXXAXXXXXMAMASAAX
SAMXMXSMMMXXXMXMASXMSMSMSSSSMXMAAMXSAMXSMXSAMXAMXAMMMMXAXSAMXXMAXAXMMMMMXAMASAXXXXAMMSSMSMSMMSMSAMXMSXMASMMXXSMXAMAMMXSSMAMMSSMSSXMXMMSAMXMM
MXSXXAXAAAXMSSXMXSAASAMXAXMXMSMMMSMSAMASXAMASXAXSSMSXMMSMMMMMXMXMXAXXXAMAXMASMMMMSXSXAXASXSAAXMMXSSSMAXAAMASMXMMMSXSXAAXSASXAXMAMMAXXAMAMXSM
ASXMMSXSMSASAAASMMMMMAMMSSMAMMXXAAMXMMASMMSAMXMMMAAXMAAXAMASXSAXMMSMMSASMXMAXXAAXMAMMMMMMASMMXSXMAXAASMSSMAXXAMAMXAXMMMXMMSMXSMAXSXSMSSSMASX
XMAMXMAXAMMMMSMMAAXAXXMXMMXAMXXMSMSSSMASAMMMMXMASAMMAMMSMSAXSAMXXAAAXAXAXAMXSSSSSXMSMMMXMAMAAASAMXSSMMAXXMASMMMSSMMMXXSMSMXMASMMXMAMAAAAMXMM
SSMMAMSMMMAAXMMMMSSMSMSSSSSMSXMAMXAAXMXSMXSSSMMAXAMXXAAMAMXSXMAMMSSSMMMMSMSMAAXXMMMAAASAMXSXMXSAMMXMAMXMXMAXAAAAAAXAXXMXAAXMASAMAMAMAMSMSASX
XAMMXMAAASXSSMASAAAXAAXAAMAAMMASXMMSMXMSAXAAAXMSMMMSMSAMXMSAXXSXAAXAAXAXAAAMXMXMMAXSXMMAXXSMSASXMMAXMMMSAMXSMMMSSMMSMASMMMXSMSMMMXAXXXAMXASX
SMMASXXXMAAMXXAMMMMMMSMMMMMMMASXSMAAAXAMXMMSMMXAASAMAASXAMSMSMAMMSSSMSSSMXMXMXXMASXMASXSMAXAXXSXSMMMXAXMAMASMAXXMMAMXMSAAAXXAMASXSMSMMMSMMMX
XAMASMSMMMXMAMMXSAMSXXAXXXMXMSAMAMMMXAMXXMXAXAMSXMASMMXSMAXXAMMAMAAAAXMXMASMSAXXXXASAMAXMSMSMASAMASMSSMSAMAMXXMAAMASMXSMMMSMAMXMAAAAAXAAAAXM
SSMMMAAXMXAMXMXAMAXSASXSSMMAXMASMMMAMSSMXSXSXXAMXSAMXXXAMAMSMSXSMMSMSMSASASAMASMMMMMXMMMXXAMMAMAMAMAAAAXXMMMSMSXMXAMAMMMSAXXXMXMXMSSSMXMXMMX
AAAMMSMSSSMSAMMMSAMMAMAXMASXXSMMXASMXAAMASMAMSSXMSAMMSMXAAMAXSAXXAAXXASASAMXMXMXAAXMAMXMMMSMMXSXMASMMMMXSAAASXSMSMMSSXMASAXXMMAAMXMAAAXSSSMS
SSMMAXXAXAXSMSMMSXSMAMXMXMMMXAXMSASXMSSMASMAMXMAMSAMMAAASMSMMXXSMXSXMAMMMXMXXXXSXMMSAXAAAXAXMAMAMASXXMXMMAMXXASASXMAXAMASMSAASASMAMSMMMAAAXA
MAMMXSMXSMMMXMMAMXSAMSXSXMAAMMSMAXMAXXXMASXMXSAMXSAMSMSMXXAXSSMXMAMXMSAMXMMMMMMSAMXSMSSSSSSSMXMAMMSXMXAXXXMSXAMMMAMASASAMAAMXMAMXAMAASAMSMMM
MAMXAAAMSASXAXMMMAMSMMAMAMMSMAAXMAXMMAXMASAMASAMXMAXAAXMXSMSMMAAMASAXXAXXAAAAAASMMXSAAAAAAMAAAXAXXXMMSMXXAAXMMMXSMMMMMMAMAMXXMMMSMSSSMSXXAMX
MSMMASXXSAMMMMMMMXSAMXAMAMXAMSSSMSSMSXXMASAMMSAMMXSSSXXMAMXMXASASAMAMMMMSSSSSMMSXMXMAMMMMMMMXMASMMAAXAAASMSMAAAXMASAAXSXMSXXSSSMAXAXMAMXMASA
AAAXXMAXMSMAMASAMXAMSSSSXXSXMAAAXAAXSAMMASAMXSXMMSMAMSXMMXAMXXXAMXSAXAAAMXMXAXAXAXAXXMXMAMAXXXAXAMXSSMSMAAAXXXSMMAMXSMMASAMSMAASMMMSMAMAXAMA
SSMMMAMXXAXXSASAXMMXAAXMAXMMMMSMMMSXXAMAXSMSAMAXXAMAMXSXSSMSAMMAMASXXXMMSAMMSMSXMMSMMMXSXSXXXMAXASXMXMAXMSMSASAXMASMMASMMSXAMSMMXAXAMMSSMMSS
XAXAXSXMSMSXMASMMSXMMSMMMSMAAXAAXXMAMSMXXSMMASAMSXMASMMMAAAXAXSXMXMMMSAAMAMAXMXAAAXAAMASMAXSSMSSMMXMASMSMAXSMMAXSXMAXMMXAMSMXMXXXMSSSXAXAXAX
XAMXMXAMXXAAMXMXAAAXMAMAAXXXSSMSXMAMXAAXAMASXMAMAASASAAXMSMMMMMXMAAAAXMXMSMMSASMMMSSMSASMAMXAAAXASMSAMXAXMXMXMAMXMMSMASMSMAMAMSMSMAMMMMSAMXS
XSMSSSSMSXSSXSAMMSXMXAMASXXSAMMAASAMMMMAASAMXSXMMXMASXMSXAAASAMAMXMMSSSMAMAMMAMXAXXXAMAXMASMMMMSMMAMASXXMSAMXXXMAMAMMMMAXSMSMSMAAMAMAAXAXMXA
AMAMAXMASAAXAMAMXMMMSMXAMAXMASXSMMAXSAMSXMAXXMMMMSMAMASXXMSMSASMXXASAAAMAMAMXMMSSMMSXMMMSMSXMAXXAMXMMSAMXSMSSSSSSSMSAXMSMXXXXMMSMSMSXXMMXMMA
MMAMMMXAMMMMXMASMAAMAMMSMMMMAMAMASAMSAAAAMSMSMMSAMMSSMSASXMMSAMAAASMMSMMSMMMXXAXAAASAAXAXXSASMSSXMSMMMAXMXXAAXAAAAASAXMAMMMSSMAMXAAXXSSXMSAS
XSXSXMMSXAXXXMAMMSMSMMMAAXSMSMXSAMAXSMMSSMXAAAAMASAAAXMAMMMAMSMMMMXAXMXMXASAMMSSSMMSSMMMXASAMAXXXXAXASMMSMMMSMMMMMMMAXMASAAAXMAMXMSAMXAAAAMM
MAMXAXAXSSSSMMAMXMAMMASMSMAAAXXMMMXMXSAMAAMAMMMSMMMXXAMXMAMASXXXXXMSMXMMXAMASAAAXXMXMMASAXMAMXMMSSMSXMMAAAMAAAAXASAMXASASMSSSMASAMXAMSMMMMXS
SAMSSMSXAXXAXSASAAMXSMSAMAMMMMMASAXMAMSSMMMMXMXSXMSSMSXMSSSMSXSMAXAAMXAAMSMMMMSAMXSSMMAXMMSXMXAAAAASXMMSXSMMMSMSASXSMMMASAAAXXASMSMMXSAMXXAX
XAMAXAXMMMSSMSASXSXAMXMAXXXAMXSAMMMMAMMMXSAMMMAXAXAMXAAXAAAASAXSXMSASXSMMMAAXXXAMXSAMMSSXXXMSXMXMMXMASXAAXASAXAMXMXMAXMAMMMSAMXMXAAXAMMMMMMM
SSMMASMMMAMXAMAMXXMAMAMSMXMXXXMASXAXXMXAASASAMSSMMSSMSSMMSMMMMMAXAMASMXAXMMMMASAMXSAMAMAXXSASXSSSMASMMMSXSAMMMAXXMMSAMMASXAXMSMMSASMMXAAMMSA
MXAMAMAXMMMMSMAMXSAMXSXMAMSMMMSAMXMMSAMMMXAXMXMAXAMAXAXMAMMMAAXMASMXMAMMMXXAMMSAMXSMMAMMXMMAMXXAASXSXXXXAMXMMSSMMSAMXSMMSAMXAAXAMXXASXSXSAMM
SSMMAMAMMSAAMSMSAXSXMXAMSMAAAAAASAMAXXXSAMSMMSMSMSXMMSMSAMMSSSSXSMSSMSAXMASAXAXAMXMASAASAMMAMXMSMMMXMASMAMAMAAMAMMMXMSAXSAMMSXSMMASMMAAAMMMX
MXASXSSMAMMSMAMMMSMSASAMXSSSMSSMSAMXSXXMAXXASAAMMXAXXAASXSAMAMXXMAMXAMMSXXAMMXMAMXMMMMXXAXSSSSMAXAAAXAXAXMSMMMSAMMSAASMMSAXMAXAXMAXXMMMMMMXX
SXMMMAMMXSAXMASAMAAAMXAMAXXAMXXMSXMMMMASMMSSMMSMASAMXAMSAMMSAMXSMSMSAXAMXMASAXSXMMXAAMSSSMAMAAXSSXSMSSSSMAXMMMSXSAMMXMMAXAMXMMMSMSMSXSXMAMAM
XASMMMSAMMMMXASASMSMXXAMMSSXMASMSMSAAXAXAAXAXXXMXMAMSMXSAMXSASXXAAAMMMMMMMAMMMMAAMSSXSAAAMMMMMMXAMXAXMAAMXMASAMXMXSSSSMSSSMSXSXAAAXSASXSASAA
XSMAASMMSAMAMMSAMAMAMSSMAAXMMMMAMASXSSSSMXSMMMMSASAMAMXSMMXXAMXMSMAMAAAXAMAMSASMSMMAMMMSMMXMAAXMASMMMMSMMASXMASXSAXMAXAXAAAAASAMXMXMAMASXSAS
SXSMMSAASASMAXMAMAMAMAMXMXSMASMAMMMSXXXAMMAXAXAMASXXASMMASAMSMSMMXSSSSSSXSXSXXXMAXMAMSMAXMAXSXMMAMAXXMAXSXMXMAMMMXMMMMSMSMMMMMMXAXAMAMMMXMAM
AAXMAXMMSXMXXXXAMXMXMMSAMMMMASXSMXMXMSXXMAASXSSMMMMSASXSAMAXMASAMMMAAAAAASMXMMMSAMXXSXSAXMMMXAAMSSMMMSMMMMSSMAMMMMSMMAAAXAMXAAXSMSMSMSXSXMAM
MAMXMXSAMASAXSSSMMSMAMSXSAAMMSAMXAMAMASXXSMMAXAXAAXMMMMMXSMMMMMAMAMMMMMMAMMAXAAMAMMSMMMMSMSAMMMMXAXMAAMAAAAMXAMAAAAAMSSSMSMSSSXMXAMAXXAMSMAS
MAMAMSMAXAMMMMAAXAAMXMMMXMMSAMAMSMXSAMAMMXAMXMXSMSMXSMSAMXAXMXSXMMSAMXAMAMXMMMXSAMXAAAXAAAMMXASXSAMMSMSMMMSXSSSSMSSSMMXXAAAXAMXXMSSSSMXMASAS
SASASXSAMXMSAMAMMSXSASASMSAMXSMMXSAMMXMSSSMSSXMAXXASAAXAAMXMAMMAAAXASXXSXMASASXSXXXSSMSMMSMSSMSXMASXMASXMAMXAMAMAAAMMSXMSMMMMMXSAXAXMASXXMXS
SAXMSXMXSAASMSMSXXAXXXASAMXXXMXSAMXMXAXAAAAMXSSSSSSSMMSXMSMSMSSMMMMXMMMMMSMSAMASXMXMMMSAAAAXAAMXSAMAMMMAMASMSMAMMMSMASAMXSAAAAAMXMMMSMASXMMM
MSMXXAXAXMMMAAAMXMMMSMMMXMSMXSAMXAAMSMMMSMMMAXXAAXAMXXMXXXAAAMXAAMMMXAAXAAXMAMSMMSAAAAMAMSSXMMMAMSMXMASXMXSAASASAAXMMMXMAMXSMMXMMAXAAXAXMXAS
AAMXMMMSSSXMSMSMMMAAAAAMAAMAXAXSSMMXAAXAAMMMSSMMMMMMMAXSMMSMSMSMMSAASXSSSSSMAMXAASXSMSSMMAAASAMXSXMASASASAMMMSXSMSXSAMXMSSXXMMSAMSMSSMMSASMS
SMSAMXAAAAMMMXMXXMMMMSMSMMMAMMXXXASMSMMSMMMAMAXSXMXMSMXAAAAMXAXXASMMMMAXXAAXMAMMMSAMAXXMMXSMMAXSAXMAMXSXMASAMXXSAMAMMSAAAMMSAASAMXAMXMAAMAMX
XXSASMMSSMMAXMASMSMXAMAXAXSAMXAMMMMAXMAAAAMASXMMXSAMAMMMMMSSMXMMMXMMSMSMMXAMXAAAAMAMSMAXMAXASMMMAMMASXMXSAMXMXMSAMSMASMMMMAMMMSAMMSMMXXSSSXS
MASXMAMAMXSASXXAAAAMSMSMMMXAMXAXSAASMMSSSMSASAMXASXSASASAMXAXAMSMMSASXMASAMXSMSMXSXMASMMMMXMXAXMAMSASASASXSXSAASMMXMAXXAXXSXSXSAMXMASXAMAMMM
MAMASXMAXAMXAMXMMMSMMAXAMAMXMSXMSAXMAXAMXXMXMXMMMSASASAMASMSMMMSAXMASASAMSAMXMAMXMASMXMXAMMXSAMSMXMXSXMAMAAASMMMAXSMXSMAMXMAMMSMMMXMMMMMAMXM
MSSXMAMMMSSMMXAXAAXAMXSXMAXSAXAXXMMSMMSSMSXXMASXMXAMMMXSAMXAXSAMMMMAMMMAMAXMASASAAXXXMASAMAMMMMXMAMMMMMSMAMXMAASMMSMMAMASMMAMXXAAAASAMXSXMAS
MAMMSSMXAAAAXMSMMXSAMXMAMAXMMSMMAAAXAAMAXXAASAMXXMXMAXMAXXMMSMAXSAMAMXSMXMASASXSMSMXSAMXSMSXSASXSAMAXAAMAMXXXSMSXXXSAAXMXAXXSSSSMSXXASXMXSMX
MXSAAAAMMSSMMAMXXXAMSMSAMSMSAMXSSMXSMMSSMMSMMASXXXSSXSMMXMAMAXAMAMSAXXAAASAMASXSXAAMASAAXXMASASASXSSSMSSMSMSXAAXXMASXSMSMSMXMMXAAMMSXMAMASAS
SAMMSSMXMAXAXSXAMXMMAMMXXXASXSXXAMMXAXXAXXMXSXMXSMAXSMXAASMMSXMSMASASAMSMXAMSMAXSXXXAXMSSMMMMXMAMAAASAMXAAASAMMMAMXMAMASAXXAMSXMMXMASMXMASMA
MASAMXMXMASXMXMASAXSASMSSMXMAMXSAMSMMMXXXMMASAMXMAAMASMSMMAXAAMMXMMMMMXMXXSMMMXMAMMMMSAAMAMXSMMSMXMSMSMMXMSMAXAXXXAMMMAMMSAMXXXMXXMAMXXMMSXM
SAMASXXMAXSXMMAXXMMSASAMXAAMAMMMXMXAAAMMMSMASASMMSXSAMXXASXMSSMSAXMMAXAXMAMAMASXXXAAAMMMMAMMXAMXMMXMAMXXMSAXSMMMSSMSSMASXMXMAXMMXMASXXSXMMMX
MASAMMASMXSAAASMMMAMXMMMMSXSASXMSASMMSAAAXMASXAAAMAMAMSXMXSAAAXSAMXSSSMSXAMAMAMMMSMMSXXXSMSASXMAAMAMMMSMMSAMMAAAMAXAASASAMSMSMSAMXAMMMXXXAAX
SMMMXSXMMASXMMMAMMAXAMMXAXASMSAXMASAAXXAMXSMSMMSXMASAMXAMAXMSSMMASXMAMMMMMMAMASAAAMAXXMXAMXXMMSSMSMSAAXSAMXMSXMMSMMSXMMSAMAAAAXXAMASXAAMSMSM
MXAMMSSMMASAXAMAMMASXSAASXMMASXXMAMMMMXMXXSAMXXMASAXMXMAMXMAMXAMAMAXMASAASXMSASMSXMASASXXXXMMMAXMAXXMSSMSSSMSAMAAXAMXMASMSMSMSMAAMMMMMMXSAAA
SMMSAXASAAXAMXSSMXMAMXMSMASMAMMMSMSAMXMMMMMMMMASMMMSAMSSMSAMXSSMASMMXMSMMSAAMASMMXMXSAMMMMMSAMXSMSSSSMAMMSXASAMSXMXSSMAXXAMXAMASXMSMSXSAMSMS
AAMMMSMMMSSMSAMXAMXMASMAXXMMMXXAAXMASAAAASASAXMAMAAMAMAAASXMMAMMXMAMAMXMXSXMSAMAAAMMMXMAXAASMSMSAAAMMMAMXXMMMXMXASAMAMASMSSMXMAMXMMAXAMMMAMA
SMMAAAAAAMAMXXXAXMAXXAXMXSXXSAMXSXMASXMMMSASXMXAMMSXSMSMMMAMXMSXSSMMAMMSAMAMMASMMXSAMXSSMMXXXAAMMMSMXXSXMMMSAMMXSMASAMXXAMMXMMAMSMMMMXMXSASX
XAXMSMXMMSAMMMSMSSMSSMASAXXXMASXAXSASAMSAMMMMXSXMAMMAAAMASAMXXAAXAMSAMAAMSAMSXMASAXMAXAMXMASMMSMSAMXMMMASAASASMXXMXMASMMSMXAMMMXSAAMSMSAXMMM
SXMMMMSAMSASAAAXAAAAASAMASXMSAMMAMMASAXMAMMAAMMAMAMXMSMSAMMXMMMSMMMSASMMXMAMXASAMMSMMXSMXSAMXAAXMASXMASMMSXSAMXMMXMAXMXAMASXSASXSXMSMAMMXSAM
MMXXAXMAMSASMSSSSSMSXSMSMMXMAASXMMMAMXMSAMSMMSSSMMMXXAAMXSSSMAAAAAAMAMXXSSSMSXMASXAXSAMXAMMMSSXXAMXMMXXSAXMMMMMXSASAMXMSMAAASASMXXMAMMMAAMAS
XXAMMMMAMXXXAAAMAXAXXMASXAXSXXMXAAMSSXAAAXAAAXXMASAMXMXMASAASMSSSMXMSMMAAAXASMSSMXASMMMMMSAXXXMSMMAXXASMMXMAAAXASAMXAXSAMXSMMXMXSMSSSMMMMSAM
MMXMAMSMSXMMMMSMMMSMSMMMMSMSMSMSMMMASMSSSMMMMSASAMASAMSSSMSMMXAMMMXXAAMSMSMMMMXXAXMAAAAAMAMMMXAAAMXSSMMXAASMSMSAMAMASMSAMMAXAASMMAAAAMXAXMXM
SMAMAXAAMAAMAAAAXAMAMXAAAXAMAAAMASMMXXAMMAXAMSAMASAAASAMXAAXSAMXAAMXMMMAMAXMXMAMXMAXSMSXSASAAXSXSAMXMAAMSXSAAAMXMXMMMASAMAMASXSAMMMSMMMMMXAM
AAMSSSMSSXSSMSSSMSMAMSSSSMAMSMSMAMAXSMSSXSMMMMXMAMMSSMXSMSMXAMXSMSXSAAMASMXMAMASXSSXMAXXSAXMSMAAMMAASMMXMAMMMMMMSMSXMMMAMMXXMASMMXAXAMSSSSSS
MXXAXSXXAAXXMAXMAAMXMMXMXMSMXAXMAXXMAAMXMXASXSAMASXXXAXXMXMSMMMXAAAMSMSXXXASXSMSAAXSSXSMMSMAAAMXXXSXSMSXSAAMASAAAAAXAXSXXXSAMXSXSMMMAMAAAAAS
XMMSSMSMMMMASXSMSMSSXSMMAXXASXMMSASAMXMAXXAMASASASMSMSMXMAMAAMMMMMSMXMAXXMAXAAXMMMMXSASXAXMMMXAMMMMAMAMAAMMSASMSMMMAMMXMXMSXMASASASMSMMMMMMM
MXAXMAXXMASXMASAAAXXAMMMXMMXMAMAAXSSSMSSSMAMXMAMAXAMAAXMXASMSMAAXAXMXSAMMMSMSMMXMASAMXMMMSMSXAXAAAMAMAMXMAAMMSAAAAXAMMAMAMXASMMMMMMAAXMASMXM
AMSSMSMSSMSAMAMSMSMMXMASMMSMSMMASMMASAAAAMMMAMXMSMMMSMMSSMSAXMSMSMXSAMASXAAXAAMXMAMASXMXAAAAMAMXSMSMSAMXMMSSMMMXSMXXAXAMAMMXAXAAAXMSMMMAXAAX
SAMXAAAAMXMXMXMXXXXMMMASAAAAASXAMAMAMSMSMMMSASMAMAAAXMAXAXMMMAXAAAAMASAMMSSSSSSMMXMXMXXASMSMMASAAASMSAMXAXXAXAMMMASXSSMSASMSMSXSSSMAMAMMSSSS
MAAMMMSMSMSMMMMAMXMMAMASMMMXMSMSXMMXMAXAXMXMAXMASXMMSAMMAMMSMAMSMMMSXMAXXXAAAAAMMAMXMSXAMXMASASMMMSASAMSSMSMMSSXMAMAAAASAAAAXAXMXMMXMASXAAAM
SSMMSXAXAAAASAMAAAASASAMAXSXXXAXXSAMXMSMSMAMSMMMSAAXMASMSAXAMXMAAMASXSAMMSMMMMMAMASAAMMMMMSAMASXSAMAMAMAAXXXMMXMMMMMMMMMAMXMSMMSAMSXSASMMMMA
XAXSASMSMSMMSAXASMMAMMXSAASXMMSSMMASXXSAAXSXXMSAXMMMMAMAXSSMSSSSSMASAMASAAAAXXSMSAMMXMSMAAMMSMMAMASXSMMXSMSXXXSAASASXXAXMASXAAAAAMAAMAMAXMXM
MMXSMMXAAXMASXMXXMXAXAXMMMMAMXAXAAAXAXMXMXXAMAMMSSMMMASASXMXMAMAMMMXXSAMMSSMSXXXAMSSMMAMMXMAAAMXMASXAXAXAMXSAASXXSAXMMASXMASMSMSMMXSMAMSXMSS
SXAMXMSMMMSMMMMSAXXMMMXXAASMMMMSSMMSMMMSMSSXMAMXAAAAXAMXSAMXMMMAMMMMXMXMXXAMXMMMMXAAASMXSAMSXSMSMASMMMXMAMAAMAMAXMXMSSMSXMXMMMXAMXAXMXSXAAAM
AAMSAMASAAAXMAMSAMXSAMSSSMMAAXMAXXMAXAXAAAAASXMMSXMMMMSMXAMXMASASAAXSASMMSAMXAASXMSSMMAAMMXMAAXXMAXAASMMSMSSMAMSMSAMMAAXXMASASMMSMXSXXMAMMAS
MSMMMSASMSMXSSXXXMAMASAAMXXXMXMXSMSXSMSMMMMMMAXXXASXXAAAXSSXSASMSXSSMAMAASMMMSMXMAMAMMMMMSAMAMMXMAXSASAAXAMXXMAXASXXXMMMASASXMAMMMMXXAMXXXAX
SAMXAMASXAAASXMASMSXMMMSMSMMSAMXSXSXSMAXSSXMSMSAXMMAMSXSXMAMMMSASMMXMMMMMMXSAAXMAMSAMSMMXMXXAXAXMAXMASMMMAMXSMMMXMMSSSXSXMXSMMMMAAAAAMSASMMS
SXMMMMASXMSMSAMASXXAMAMAASAASASASAMASMMMAMAMAMXMXXAMXXXMXMAMSAMXMASMMAAAXAAMXSMMMXMMMAAAXMASMSSMMMSMAMAXSXMAXXMXSAXAAMMMAMASAASMSASXSASAMXSM
MMSMSMASXMXXXAMAMASXMASMSSMMSAMXMAMAMAMAAMAMAXSXXMAXASMMXSAMSXSXXXMAXSMSMMSSMMMXMAAAXSSMXMASAAMXXMAXMMMMMAMMSASAAXMMSMAXAMASMMSAXXMAMAMXMMAX
AMAAXMXMMXMASMMXSAXAXAMMAMXAMXMXMSMMSAMXMSSSMXSAASXMXSAAAXAMSAMMSMSSMMAMAMAXAAAAMMMMMXAMMMASMAMMXMSMSMSASXMAMMMMSXAXXXSSSMASAXMAMAMMMXMASXMX
XSMSMSAAMSMAMAAXMASMMSSMAMAMSAMXAMAXMXSXAAAAMAMAMASXXXMMSMMMMAMAAXAMAMAMSXMXSMSSSSXSXXAMXMASXMMXSSMAAASAMMMMXXAMXMSXMAMAAMAXMMMAMAMXSXSASASX
ASAMASMMMAMASXMXAAAAAXAMSSSMMAXXMMAMAASMMMSMMASAMMMMSMSXXASMSMMSSSMMMSAMXASAMAMAAAASAMSMSSMMAXAMXAMMMMMXMASAMSXSXAXMASMSMMSSMMXXSASAMAMXSASM
MMAMMMSSSMSASMMXMXSMMSSMMAAMSMMSMSMMMAMAXAAASXSMSXAAMXMASAMAAAMAMAXMASMMSAMXMAMMMMMMAMAAXMASMMSMSMMAXAXASMMXMXAMMSMSMMXXAAAMAXSASASAMXMXMAMA
XSXMXAAXAAMAMMSASXXAAAAAMSMMAAXAASMMXASMMMXXMAMXXMMMSAMXMAMSSSMXMXMMAMAMAAMXSMSXXAXMXMMSMMMMMAAASXSMSMMMSXXXSMXMAMMSAMAXMMMMAMMAMASAMXSSMAMS
XMAMMMMMMXMSMXMASXMMMMSMMAMSXSMMSMASXMSASMSSMAMMMSAASAMXSXMAMAMSSSXMASMAMXXAAXAMSMXMAXAXMMAAMMMSMAXXAAASMMSAAASXXSAMAMXMXASMSMMAMASAMXAAMMSM
XXAMXAXSAMXAMMMSMXMAMXMAXAMMMMMXMMMMAAMAMAAXSMSMAMMXXAMXMAMASAMXAMSSXSXMXMMSSMMMAAASAMXSXMXSSMMXMMMSMSMSAASXMMMAAMASAMXSMASAAXSXMXSXMASXMSMM
MMSXMASXASXMSAMAAXSAMSMSSSSSXMASMASXSMMMMMXMMSXMMXXSXSMAMAMMSMMMAMASAMXXAXMAXAAXMXMSAMXXAMSMXXXMASAMXXMMXMSAMXMMMMXMASAMMAMMMXMAMAMXXMASMAAA
AXAAMSMMMMAAMAMMXMAAXAAMAXAXAAAMMXSAAMAAAMXMXAMMSMMXAMSMSXSMMASMXMASXSMSSSMSSXMSSMXSAMSMSMAASMSSXMAXAXXSMXSXMAMAAXXSXMXSMXSXSASAMMSMMXMAXMMM
SMMSMAAXXSXMMSMMASMMSMSMXMASXMASXSMMMSSSXSAMSMXAAAXMXMAXAASASAXXMMASAMAAAAAMAAMAMSAMMMSAAMMSMAAMMXMMMSMMMXXSSXSSXSAMXMMSAMXASXXAXAAASAMMSSXX
XAXXMSSMMMASAXASASXAAMXXXAXMASXXXAAAXAAXAMMXAAMSXSMAASMMMSMAMXSAMXAMAMMMSMMMSMMASMXXSAMXMSMAMXAAAASAMXAAMMMXXXAXASXSXMAMAXMMMMMMMSSXSASAAXXA
MMMXMXXAAMXMASAMAXMXSMMSSMXXAMMSSSMMSMXMSMMSMSMMAMMSMSAASMMSMASAXAMSXMAMAMXXAAXSMMAAMMMAMAXXSMSMSMMAXXMMSASMMAMMAMXXAMSSSMXMASAMAMMMMMMMMSSX
MAAAMMSMMMAMAMXMSMMAXAXAAAMMMSAAMXAXAMAMXMAAMAMXAXAAXXMMSAAXMXSXMXXXASMSSSMMSSMAAMSSSSSSSSSXSAMAMXSMMASASMSAMXXMAMXSAMXAMMASASAMMMASMSMXSAXS
SASXSAMAASAMSXMAXAXASAMSSMAAAMMSSSMMASASAMSXSASMSMSMSMSMMMMSMMMXAXMSAMAAAXAXAMXSXMXAMAAAAASAMXMMXMASMMMASMSMMMXXXSAMSAMXMSASXXXMXSAMAAAMMSMA
AAMMMASAMSASMAMSSMMMMMAMAMSSXSAAXAASXMAMXMAXMASXMAXAMAXXASAMXXAAXMXAAMMMSXXMASAXASMAMMMMMMMXMMMMMAAASXMAMASASAMSXMASAMXAMMMXAXSAMAXMSMMMAXXS
MXMXSAMXXXMAMAMXAXSAAMSSSMAMXMMSMSMMSMXMSMMMSAMAMSMSMAMXAMXSAMXSMSMSSMXAMMSSMMASAXXXMASXXASMAMASMMSAMXMAXXSAMSAAXSXMMXSXMASMAMMAMAXXXAMMASMA
SMMXMASXSSMASXSSMMXXMSAAAMSSSXXAAMXMAMAMXAAAMASXMXAXMAMXAXAMMAAXAAAXAMXXAAAAAXAMXMMSXMMSAMXAASMSAAXAMAMMXMMXMMSMMMMMMAMASASMMXSMMMMAMMMMAXAM
AXXAXAXAMASASAMXMAXSSMMSMMMAMAMMXMAMMSSSSXMMSMMMAMXMSASMMMMSMSSSSMSMSXMMMMSSMMXSXAXMASAMMMMXMXMMMMSSMSASASXSAXXXMXAAMXSAMMSAXAXXAXMASASMSSSX
MSXMXXSASAMXSAMXXAXMAMMAAAMAMMMSMSSMXMAAMMXAXAAAXMXAMMXXAAAAAXAAAAXMMAXXAAMAMSMMMSAMSMXXAAMMXSAXAAAAAXASASASXMXMMMSSSXMMXXXMMMSMXXSASXSAAAMA
XMASAMSAMMXAMAMXMMXMAMSMSXSAXXAXMAMXMSMAMSMMSSSMSAMXMMMMSMMMSMMMMMMSXSAMXMMAMXAAAMMXXAMMSXSAASAMSXMMMMAMAMMMAMMSAAMAXAASXSMXAMAXSMMMSAMMMSMS
MXAMAXMAMSMMSSMMSASXMMAXMMMMMMMSMASAMMSAAMAMAAAXSMSMSAMXMMXXMASAAXXMAXMMMXMAMXMMSMSAMAMMAAMMMMAMXAXMMMXMAMASXMAXMSSMSSMMAAXXXMXXSAAAMMMXMAXA
MMSSXMSAAXAMAAAXMASASMMSXAAAXAAAXXSASMXXMMAMMXMMMAAAMXMAASXMSSSXSSSMAMXAAAMAMXXMMAMMXMMMMMMAXSXMSMMMMMMSMSAXAMMSMAAMAMMMSMMMSAMXSXMMSMSAMXXM
XAMAMASMSXXMXSMMMAMXAMAMMSMSXXMMSMSMMXMASMMXSAXAMXMSMXSASXMASXMAMAMMAMSSSMXAXSMSMAMXXSXAASXMXSSXAXAMXAXAAMASXSXAMSSMAMMAAAXMAXMMSXSMSAMXSAMX
MMSMMAXXMAXSMXMAMXSAMXAMMXSXXSMXXAXAAMMASAMAMXSMSAAAAXSMMAMXMAXXMAMSXMMAAMMSMSAAXAMSXMAMMMAMMMMSMSSMSASMMMMAAXMMXXXMAMMSSSMMSMSXMASAMXMAXASM
MAAAMXSAMMMSAMSMSAMAXSXSXXASAMMASMMMMSMASXMASXMAAMMMSMSASXMASMMXMAMMMMMMXMAXAMSMMSSMAMASMSSMAAAXMAMXAAAXASMMXMMSAMSSMSMAAMASMAXAMAMMMMMMXAMA
MSSSMASAASXMAMMXMASAMMXXXAMMSMAASAAXSXMASASASAMAMXSXAAMAMSAMXAAXSMSAAXASMSAMXMASAXAMXAASMAASXMSSMASAMXMAMSAXASXAASAAXAXMASXMMXMMMMSAAXAAMSXS
XXAXMASXMMMSAMXAXMAMSAMXASXSMMMSSMMMMSMASMMXSMMAXMASMSMXMXMSAMXXXASMSSMSXMASASMSXSAMXMMSMSMMXAXXXASXXMASMSXMASXSSMXSMMXXMAMXXXXXSSSMSSXXXMAX
"""


In [ ]:
check_xmas(major_input)


2644

# --- Part Two ---

The Elf looks quizzically at you. Did you misunderstand the assignment?

Looking for the instructions, you flip over the word search to find that this isn't actually an XMAS puzzle; it's an X-MAS puzzle in which you're supposed to find two MAS in the shape of an X. One way to achieve that is like this:
```
M.S
.A.
M.S
```
Irrelevant characters have again been replaced with . in the above diagram. Within the X, each MAS can be written forwards or backwards.

Here's the same example from before, but this time all of the X-MASes have been kept instead:
```
.M.S......
..A..MSMS.
.M.S.MAA..
..A.ASMSM.
.M.S.M....
..........
S.S.S.S.S.
.A.A.A.A..
M.M.M.M.M.
..........
```
In this example, an X-MAS appears 9 times.

Flip the word search from the instructions back over to the word search side and try again. How many times does an X-MAS appear?


# thoughts

how many x:mas's are there?

```
m.m
.a.
s.s
```

```
s.m
.a.
s.m
```

```
s.s
.a.
m.m
```

```
m.s
.a.
m.s
```




In [ ]:
def find_x_mas(x, y, message, x_max, y_max, x_string):
    x_string_len = len(x_string)
    x_string_leg = int(x_string_len / 2)
    out = []
    instructions = {
        "ne": [[-1, -1], [0, 0], [1, 1]],
        "nw": [[1, -1], [0, 0], [-1, 1]],
    }

    for i, v in instructions.items():
        # print(x,y,i,v)
        res = "".join([message[x + v[j][0], y + v[j][1]] for j in range(len(v))])
        # print(res)
        out.append(res)
    # print("message group: \n", message[x - 1 : x + 2, y - 1 : y + 2])
    # print(out)
    for i in out:
        if not (i == x_string or i == x_string[::-1]):
            # print("no", i == x_string, ":", i == x_string[::-1])
            return 0
    # print("yes")
    return 1


def gen_matrix(text):
    rows = text.splitlines()
    tem = []
    for i in [r for r in rows if r]:
        tem.append([j for j in i])
    return np.asmatrix(np.array(tem))


def check_x_mas(text_input, x_string="MAS"):
    x_string
    x_min = 1
    y_min = 1
    a = gen_matrix(text_input)
    s = a.shape
    x_max = s[0] - 1
    y_max = s[1] - 1
    acc = 0
    for x in range(x_min, x_max):
        for y in range(y_min, y_max):
            acc = acc + find_x_mas(x, y, a, x_max, y_max, x_string)
    return acc


check_x_mas(test_text)


9

In [ ]:
check_x_mas(major_input)


1952

In [ ]:
a = gen_matrix(test_text)
b = "mas"

x = 2
y = 1
x_min = x - 1
x_max = 3 + x - 1
print(x_min, x_max)

a[x_min:x_max, 0 + y - 1 : 3 + y - 1]


1 4


matrix([['M', 'S', 'A'],
        ['A', 'M', 'X'],
        ['M', 'S', 'A']], dtype='<U1')